In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import blueice as bi
from laidbax import base_model
data_dir = base_model.THIS_DIR + '/data/'

In [2]:
# Load the published result
published = pd.read_csv(data_dir + 'sr0_result.csv').loc[10]

# Load the data
data = pd.read_csv(data_dir + 'sr0_search_data.csv')
data['cs1'] = data['cs1_pe']
data['cs2'] = data['cs2_bottom_pe']

# Remove the silly outlier. Would make limit very dependent on MC statistics of WIMP model
# (If I recall for SR0 we were quite lucky and had a bit of an underfluctuation there...)
# And coincidentally or not, this also reproduces the published limit much more accurately..?
data_nooutlier = data[~ ((data['cs1'] > 68) & (data['cs2'] < 1e3))]

In [3]:
ll = bi.UnbinnedLogLikelihood(base_model.config)
ll.add_rate_parameter('wimp')
ll.add_rate_parameter('er')
ll.prepare()
ll.set_data(data_nooutlier)

In [4]:
limit_options = dict(bestfit_routine='scipy',
                     minimize_kwargs=dict(method='Powell',))
limit = ll.one_parameter_interval('wimp_rate_multiplier', bound=1,  **limit_options)
limit, limit/published.limit_zb

(0.08518443798073073, 1.0179264750930634)

In [5]:
n_trials = int(1e3)

limits = np.zeros(n_trials)
for i in tqdm(range(n_trials)):
    d = ll.base_model.simulate(rate_multipliers=dict(wimp=0))
    ll.set_data(d)
    limits[i] = ll.one_parameter_interval('wimp_rate_multiplier', bound=1, 
                                          **limit_options)
    
sensitivity = np.median(limits) 

100%|██████████| 1000/1000 [00:47<00:00, 21.11it/s]


In [6]:
sensitivity, sensitivity/published.sensitivity_median_zb

(0.082309944336309296, 0.93753609141886318)